In [3]:
#Import required libraries
import os
import csv
import numpy as np
import pandas as pd
from bioservices import PSICQUIC
from bioservices.uniprot import UniProt
u = UniProt(verbose=False)

metaDir = os.path.join(os.path.dirname('__file__'), '..', 'metadata')

# Genes list for Crohn's disease
Genes = ['ATG16L1','IL23R', 'IRGM', 'ADAM30', 'CPEB4', 'CREB5', 
         'FASLG', 'FUT2', 'GPX4', 'HMHA1', 'IFNAR1', 'IFNGR2', 
         'IL31RA', 'IL6ST', 'JAZF1', 'LACC1', 'LGALS9', 
         'NOD2', 'NOS2', 'PTPN22', 'RASGRP1', 'RIPK2', 
         'SP140', 'SPRED1', 'TAGAP', 'TNFSF18', 'UCN']

s = PSICQUIC(verbose=False)

# Intersection for 2 lists
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

gdf = pd.read_excel(os.path.join(metaDir, 'Gene_Map.xlsx'), sheetname='Map')
  
    
# Getting the Gene Map
gene_id_list = list(gdf['GeneID'])
gene_list_master = list(gdf['Gene'])

['ATG16L1', 'UCN', 'TNFSF18', 'TAGAP', 'SPRED1', 'SP140', 'IL23R', 'IRGM', 'ADAM30', 'CPEB4', 'CREB5', 'FASLG', 'FUT2', 'GPX4', 'HMHA1', 'IFNAR1', 'IFNGR2', 'IL31RA', 'IL6ST', 'JAZF1', 'LACC1', 'LGALS9', 'NOD2', 'NOS2', 'PTPN22', 'RASGRP1', 'RIPK2']


In [4]:
'''Collect Data for BioGrid Database'''
def return_gene_id(gene_string):
    gene_string = gene_string.replace('entrez gene/locuslink:','')
    return gene_string


gene_list = []
for gene in Genes:
    gene_list.append((str(gene)+" AND species:9606"))
df = pd.DataFrame(columns=('Gene ID', 'Seed_Gene','GeneA','GeneB'))
c=0
j=0
for gene in gene_list:
    data = s.query("biogrid", gene)
    for row in data:
        if(len(row) >= 2):
            df.loc[c] = [j,Genes[j],return_gene_id(row[0]),return_gene_id(row[1])]
            c+=1
        
    j+=1
    
df.to_csv('BioGrid.csv',encoding='utf8', index=False)

In [5]:
df = pd.read_csv('BioGrid.csv')
df = df[['Seed_Gene','GeneA','GeneB']]

df = df.set_index('Seed_Gene')

bioList = []
bioList = df["GeneA"]
bioList = list(set(bioList.append(df["GeneB"])))

bioGeneList = []

for gene_id in bioList:
    if gene_id in gene_id_list:
        bioGeneList.append(gene_id)
            
no_seed_gene_in_Bio = len(bioGeneList)

a=set(df['GeneA'])

a=list(a)

b=set(df['GeneB'])

b=list(b)

a = np.asarray(a,dtype=np.character)
b = np.asarray(b,dtype=np.character)

lis = []

for element in b:
    if element not in a:
        lis.append(element)
b=np.asarray(lis,dtype=np.character)
no_interacting_proteins = len(a)+len(b)

# @TODO: Need to report intercom between non-seed genes
print('Total Number of Seed Genes found in BioGrid : ',no_seed_gene_in_Bio)
print('Total Number of Interacting Proteins including Seed Genes : ',no_interacting_proteins)
print('Total Number of Interactions found in BioGrid : ',len(df))


Total Number of Seed Genes found in BioGrid :  23
Total Number of Interacting Proteins including Seed Genes :  713
Total Number of Interactions found in BioGrid :  1132


In [6]:
# Collecting data from IID data base

df = pd.read_csv('IID.csv')

iiDList = []
iiDList = df["symbol1"]
iiDList = list(set(iiDList.append(df["symbol2"])))
iiDGeneList = []

for gene_id in iiDList:
    if gene_id in gene_list_master:
        iiDGeneList.append(gene_id)
            
no_seed_gene_in_Bio = len(iiDGeneList)

a=set(df['symbol1'])
a=list(a)
b=set(df['symbol2'])
b=list(b)
a = np.asarray(a,dtype=np.character)
b = np.asarray(b,dtype=np.character)
lis = []
for element in b:
    if element not in a:
        lis.append(element)
b=np.asarray(lis,dtype=np.character)
no_interacting_proteins = len(a)+len(b)

# @TODO: Need to report intercom between non-seed genes
print('Total Number of Seed Genes found in BioGrid : ',no_seed_gene_in_Bio)
print('Total Number of Interacting Proteins including Seed Genes : ',no_interacting_proteins)
print('Total Number of Interactions found in BioGrid : ',len(df))


Total Number of Seed Genes found in BioGrid :  26
Total Number of Interacting Proteins including Seed Genes :  2379
Total Number of Interactions found in BioGrid :  3844
